# Part 2: Training an RBM *with* a phase

## Getting Started

In this tutorial we work through a simple example for full quantum state tomography of a complex wavefunction with two qubits. 
First we load the packages needed

In [1]:
import torch
import numpy as np
import pickle

from qucumber.nn_states import ComplexWavefunction

from qucumber.callbacks import MetricEvaluator

import qucumber.utils.training_statistics as ts
import qucumber.utils.data as data              
import qucumber.utils.cplx as cplx              
import qucumber.utils.unitaries as unitaries    

## Training

The data set comprises the measurements \verb|'qubits_train_samples.txt'|, the unitaries that have been applied before the measurement \verb|'qubits_train_bases.txt'| and the actual wavefunction $| \psi \rangle$ the measurements have been sampled from \verb|'qubits_psi.txt'|.

Load the files the following way:

In [2]:
train_samples_path = 'qubits_train_samples.txt'
train_bases_path   = 'qubits_train_bases.txt'
bases_path         = 'qubits_bases.txt'
psi_path           = 'qubits_psi.txt'

train_samples,target_psi,train_bases,bases = data.load_data(train_samples_path, 
                                                            psi_path, 
                                                            train_bases_path, 
                                                            bases_path)

To construct a **ComplexWavefunction** neural network state we need to create a dictionary that contains the unitaries that have been applied during the measurements. In our case this were the unitaries $\mathbb{1}$, $H$, $K$.

In [3]:
unitary_dict = unitaries.create_dict()
'''If you would like to add your own quantum gates from your experiment to 
   "unitary_dict", do:
   unitary_dict = unitaries.create_dict(name='your_name', 
                                        unitary=torch.tensor([[real part], 
                                                              [imaginary part]], 
                                                             dtype=torch.double)
                                                             
   For example: 
   unitaries = unitary_library.create_dict(name='qucumber', 
                                           unitary=torch.tensor([ [[1.,0.],[0.,1.]] 
                                                                  [[0.,0.],[0.,0.]] ], 
                                                                dtype=torch.double))
                                                                                             
   By default, unitary_library.create_dict() contains the idenity matrix and the 
   hadamard and K gates with keys Z, X and Y, respectively.
'''

The number of visible units \verb|nv| is equal to the number of qubits or the length of a tarining sample (\verb|train_sample|). The number of hidden units \verb|nh| we set equal to the number of visible units.

One may also choose to run this tutorial on a GPU by adding in "gpu = True" as an argument to **ComplexWavefunction**. 

In [3]:
nv = train_samples.shape[-1]
nh = nv

nn_state = ComplexWavefunction(num_visible=nv, num_hidden=nh, unitary_dict=unitary_dict, gpu=False)

Now we can specify the training parameters:

1. **epochs**: the number of epochs, i.e. training cycles that will be performed.
2. **batch_size**: the number of data points used in the positive phase of the gradient.
3. **num_chains**: the number of data points used in the negative phase of the gradient.
4. **CD**: the number of contrastive divergence steps; CD=1 seems to be good enough in most cases
5. **lr**: the learning rate.
6. **log_every**: how often you would like the program to update you during the training; say we choose 10 - that is, every 10 epochs the program will print out the fidelity. This parameter is required in the *MetricEvaluator*.

In [4]:
epochs     = 700
num_chains = 10
batch_size = 50
CD         = 2
lr         = 0.01
log_every  = 100

To evaluate how the RBM is training, we will compute the full KL divergence and the fidelity between the true wavefunction of the system and the wavefunction the RBM reconstructs.
This can be done by initializing the parameters of the **ComplexWavefunction** and the **MetricEvaluator**.

In [5]:
nn_state.space = nn_state.generate_hilbert_space(nv) # generate the entire visible space of the system.
callbacks      = [MetricEvaluator(log_every,{'Fidelity':ts.fidelity,'KL':ts.KL},target_psi=target_psi,bases=bases,
                                  verbose=True, space=nn_state.space)]
# The "verbose=True" argument will print the parameters in { } as a function of the training process.

After initializing everything we can start the training.

In [6]:
nn_state.fit(train_samples, epochs, batch_size, num_chains, CD,
       lr, input_bases=train_bases, progbar=False, callbacks=callbacks)

Epoch: 100	Fidelity = 0.908416	KL = 0.044572
Epoch: 200	Fidelity = 0.963343	KL = 0.020658
Epoch: 300	Fidelity = 0.977993	KL = 0.014376
Epoch: 400	Fidelity = 0.984328	KL = 0.011163
Epoch: 500	Fidelity = 0.985614	KL = 0.010323
Epoch: 600	Fidelity = 0.988434	KL = 0.008806
Epoch: 700	Fidelity = 0.988839	KL = 0.008166


### After Training 

After the training we can calculate state fidelity, observables or sample from the complex wavefunction
the same way we did from the real-positive wavefunction. However, one has to keep in mind that the sampling only works in the Z basis.

To sample from a trained complex wavefunction we define the number of samples *num_samples* we want to draw and the number of contrastive divergence steps *CD*. 

In [7]:
num_samples = 2000
CD          = 200

samples = nn_state.sample(num_samples, CD)

Also analogous to the positiv-real wavefunction we can save and load the RBM parameters and the newly generated samples using the *save* function within the ComplexWavefunction object and save additional quantities like e.g. *the samples* to the same file with the *metadata*.

In [8]:
nn_state.save('saved_parameters.pkl', metadata={'Samples':samples})